# L2: DSPy Programming - Signatures and Modules

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [ ]:
from helper import get_openai_api_key
openai_api_key = get_openai_api_key()

import os

os.environ["OPENAI_API_KEY"]  = get_openai_api_key()

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.</p>

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

### Set up API key

In [ ]:
from helper import get_openai_api_key
openai_api_key = get_openai_api_key()

import os

os.environ["OPENAI_API_KEY"] = get_openai_api_key()

### Configure the LM

In [ ]:
import dspy
dspy.settings.configure(lm=dspy.LM("openai/gpt-4o-mini"))

## Use DSPy built-in Module to Build a Sentiment Classifier

In [ ]:
class SentimentClassifier(dspy.Signature):
    """Classify the sentiment of a text."""

    text: str = dspy.InputField(desc="input text to classify sentiment")
    sentiment: int = dspy.OutputField(
        desc="sentiment, the higher the more positive", ge=0, le=10
    )

In [ ]:
str_signature = dspy.make_signature("text -> sentiment")

### Create a Module to Interact with the LM

In [ ]:
predict = dspy.Predict(SentimentClassifier) 

In [ ]:
output = predict(text="I am feeling pretty happy!")
print(output)

In [ ]:
print(f"The sentiment is: {output.sentiment}")
print(f"The sentiment is: {output['sentiment']}")

In [ ]:
dspy.configure(lm=dspy.LM("openai/gpt-4o"))
print(predict(text="I am feeling pretty happy!"))

In [ ]:
dspy.configure(lm=dspy.LM("openai/gpt-4o-mini"))

### Wait, Where is My Prompt? 

In [ ]:
dspy.inspect_history(n=1)

### Try a Different Built-in Module

In [ ]:
cot = dspy.ChainOfThought(SentimentClassifier)

output = cot(text="I am feeling pretty happy!")
print(output)

In [ ]:
dspy.inspect_history(n=1)

### Use a Different Adapter

In [ ]:
dspy.configure(adapter=dspy.JSONAdapter())

In [ ]:
print(cot(text="I am feeling pretty happy!"))
dspy.inspect_history(n=1)

## Build a Program with Custom Module

In [ ]:
class QuestionGenerator(dspy.Signature):
    """Generate a yes or no question in order to guess the celebrity name in users' mind. You can ask in general or directly guess the name if you think the signal is enough. You should never ask the same question in the past_questions."""
    past_questions: list[str] = dspy.InputField(desc="past questions asked")
    past_answers: list[bool] = dspy.InputField(desc="past answers")
    new_question: str = dspy.OutputField(desc="new question that can help narrow down the celebrity name")
    guess_made: bool = dspy.OutputField(desc="If the new_question is the celebrity name guess, set to True, if it is still a general question set to False")


class Reflection(dspy.Signature):
    """Provide reflection on the guessing process"""
    correct_celebrity_name: str = dspy.InputField(desc="the celebrity name in user's mind")
    final_guessor_question: str = dspy.InputField(desc="the final guess or question LM made")
    past_questions: list[str] = dspy.InputField(desc="past questions asked")
    past_answers: list[bool] = dspy.InputField(desc="past answers")

    reflection: str = dspy.OutputField(
        desc="reflection on the guessing process, including what was done well and what can be improved"
    )

def ask(prompt, valid_responses=("y", "n")):
    while True:
        response = input(f"{prompt} ({'/'.join(valid_responses)}): ").strip().lower()
        if response in valid_responses:
            return response
        print(f"Please enter one of: {', '.join(valid_responses)}")

class CelebrityGuess(dspy.Module):
    def __init__(self, max_tries=10):
        super().__init__()

        self.question_generator = dspy.ChainOfThought(QuestionGenerator)
        self.reflection = dspy.ChainOfThought(Reflection)

        self.max_tries = 20

    def forward(self):
        celebrity_name = input("Please think of a celebrity name, once you are ready, type the name and press enter...")
        past_questions = []
        past_answers = []

        correct_guess = False

        for i in range(self.max_tries):
            question = self.question_generator(
                past_questions=past_questions,
                past_answers=past_answers,
            )
            answer = ask(f"{question.new_question}").lower() == "y"
            past_questions.append(question.new_question)
            past_answers.append(answer)

            if question.guess_made and answer:
                correct_guess = True
                break

        if correct_guess:
            print("Yay! I got it right!")
        else:
            print("Oops, I couldn't guess it right.")

        reflection = self.reflection(
            correct_celebrity_name=celebrity_name,
            final_guessor_question=question.new_question,
            past_questions=past_questions,
            past_answers=past_answers,
        )
        print(reflection.reflection)


In [ ]:
celebrity_guess = CelebrityGuess()

In [ ]:
celebrity_guess()

## Save and Load

In [ ]:
celebrity_guess.save("dspy_program/celebrity.json", save_program=False)

In [ ]:
celebrity_guess.load("dspy_program/celebrity.json")

In [ ]:
celebrity_guess.save("dspy_program/celebrity/", save_program=True)

In [ ]:
loaded = dspy.load("dspy_program/celebrity/")

In [ ]:
loaded()

In [1]:
import os
import zipfile

# Step 1: Move to the parent directory
os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
parent_dir = os.getcwd()

# Step 2: Loop through each item in the parent directory
for item in os.listdir(parent_dir):
    item_path = os.path.join(parent_dir, item)
    
    # Only zip directories
    if os.path.isdir(item_path):
        zip_filename = f"{item}.zip"
        with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, dirs, files in os.walk(item_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, item_path)
                    zipf.write(file_path, arcname)
        
        print(f"✅ Zipped: {zip_filename}")


KeyboardInterrupt: 

In [2]:
import os
import zipfile

# Step 1: Move to the parent directory
os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
parent_dir = os.getcwd()

# Step 2: Loop through each item in the parent directory
for item in os.listdir(parent_dir):
    item_path = os.path.join(parent_dir, item)
    
    # Skip directories named 'models'
    if item == "models":
        print(f"⏭️ Skipped directory named 'models': {item_path}")
        continue

    # Only zip directories
    if os.path.isdir(item_path):
        zip_filename = f"{item}.zip"
        with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, dirs, files in os.walk(item_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, item_path)
                    zipf.write(file_path, arcname)
        
        print(f"✅ Zipped: {zip_filename}")

✅ Zipped: .jupyter.zip


KeyboardInterrupt: 

In [9]:
# list with pathlib (object oriented)
from pathlib import Path
for p in sorted(Path('.').iterdir()):
    print(p.name, 'dir' if p.is_dir() else 'file')

.dockerenv file
bin dir
boot dir
bootstrap.sh file
dev dir
etc dir
home dir
lib dir
lib64 dir
media dir
mlartifacts dir
mlruns dir
mnt dir
opt dir
proc dir
requirements.txt file
root dir
run dir
sbin dir
srv dir
start.sh file
sys dir
tmp dir
usr dir
var dir
wheels dir


In [8]:
os.chdir('..')

In [12]:
import os
from pathlib import Path

# Get the current notebook's directory
notebook_dir = Path().resolve()
os.chdir(notebook_dir)

print(f"Changed to: {os.getcwd()}")

Changed to: /


In [11]:
!pip install ipynbname

/bin/sh: 1: pip: not found


In [13]:
import os
from pathlib import Path

# Get the directory of the current file (notebook)
current_file_dir = Path(__file__).parent.absolute()
os.chdir(current_file_dir)

print(f"Changed back to notebook directory: {os.getcwd()}")

NameError: name '__file__' is not defined

In [14]:
import os
from IPython import get_ipython

# Get the full path of the current notebook
notebook_path = get_ipython().run_line_magic('pwd', '')  # Gets current notebook directory

# Change back to that directory
os.chdir(notebook_path)

# Confirm it's changed
print("Current directory:", os.getcwd())



Current directory: /


In [ ]:
import os
import json
import requests
from notebook import notebookapp

# Find the notebook path via the Jupyter server
servers = list(notebookapp.list_running_servers())
if servers:
    server = servers[0]
    response = requests.get(f"{server['url']}api/sessions", params={'token': server.get('token', '')})
    sessions = json.loads(response.text)
    for sess in sessions:
        if 'notebook' in sess:
            notebook_path = os.path.join(server['notebook_dir'], sess['notebook']['path'])
            notebook_dir = os.path.dirname(notebook_path)
            os.chdir(notebook_dir)
            print("Changed back to notebook directory:", notebook_dir)
            break
